## Implementing VGG Blocks

In [7]:
from keras.layers import Input
from keras.layers import Conv2D, MaxPool2D
from keras.models import Model
from keras.utils import plot_model

# function that will create vgg block where the input is passed with the number of filters and convolutions.
# function will add the convolution layers to input layer with the given filter
# max pooling is done by the end of the convolutions
# return the layer which was convoluted and max pooled
# input parmeters:
#  (a): input_layer : initialised input
#  (b): n_filter    : required number of filters
#  (c)  n_conv      : number of convolution layers to be added
def vgg_block(input_layer, n_filter, n_conv):
    for _ in range(n_conv):
        # here we are using 3x3 filters which is by standard used in VGG
        input_layer = Conv2D(n_filter, (3,3), padding = 'same', activation='relu')(input_layer)
    # max pooling is done by the end of the convolution layers
    input_layer = MaxPool2D((2,2), strides=(2,2))(input_layer)
    return input_layer

# initialising input layer
visible = Input(shape=(256,256,3))

# returning the updated layer with added convolutional layers
layer = vgg_block(visible, 64, 2)

model = Model(inputs= visible, outputs = layer)

model.summary()

plot_model(model, show_shapes=True, to_file='vgg_block.png')



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 256, 256, 64)      1792      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 128, 128, 64)      0         
Total params: 38,720
Trainable params: 38,720
Non-trainable params: 0
_________________________________________________________________


## Model with multiple VGG blocks 

In [9]:
# function that will create vgg block where the input is passed with the number of filters and convolutions.
# function will add the convolution layers to input layer with the given filter
# max pooling is done by the end of the convolutions
# return the layer which was convoluted and max pooled
# input parmeters:
#  (a): input_layer : initialised input
#  (b): n_filter    : required number of filters
#  (c)  n_conv      : number of convolution layers to be added
def vgg_block(input_layer, n_filter, n_conv):
    for _ in range(n_conv):
        # here we are using 3x3 filters which is by standard used in VGG
        input_layer = Conv2D(n_filter, (3,3), padding = 'same', activation='relu')(input_layer)
    # max pooling is done by the end of the convolution layers
    input_layer = MaxPool2D((2,2), strides=(2,2))(input_layer)
    return input_layer


visible = Input(shape=(256,256,3))

layer = vgg_block(visible, 64, 2)

layer = vgg_block(layer, 128, 2)

layer = vgg_block(layer, 256, 4)

model = Model(inputs=visible, outputs=layer)
model.summary()
plot_model(model, show_shapes=True, to_file='multiple_vgg_blocks.png')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 256, 256, 64)      1792      
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 128, 128, 128)     147584    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 64, 64, 128)       0         
__________

## Naive Inception Module

In [15]:
from keras.layers.merge import concatenate

def naive_inception_module(input_layer, f1, f2, f3):
    # 1x1 convolution
    conv1 = Conv2D(f1, (1,1), padding='same', activation='relu')(input_layer)
    # 3x3 convolution
    conv3 = Conv2D(f2, (3,3), padding='same', activation='relu')(input_layer)
    # 5x5 convolution
    conv5 = Conv2D(f3, (5,5), padding='same', activation='relu')(input_layer)
    # 3x3 maxpooling
    pool = MaxPool2D((3,3), strides=(1,1), padding='same')(input_layer)
    
    # concatenate all the layers
    output_layer = concatenate([conv1, conv3, conv5, pool], axis=-1)
    
    return output_layer

visible = Input(shape=(256,256,3))

output = inception_module(visible, 64, 128, 32)

model = Model(inputs=visible, outputs=output)

model.summary()

plot_model(model, show_shapes=True, to_file='naive_inception_module.png')
    

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_32 (Conv2D)              (None, 256, 256, 64) 256         input_10[0][0]                   
__________________________________________________________________________________________________
conv2d_33 (Conv2D)              (None, 256, 256, 128 3584        input_10[0][0]                   
__________________________________________________________________________________________________
conv2d_34 (Conv2D)              (None, 256, 256, 32) 2432        input_10[0][0]                   
__________________________________________________________________________________________________
max_poolin

## Inception Module

In [21]:
def inception_module(input_layer, f1, f2_in, f2_out, f3_in, f3_out, f4_out):
    # 1x1 convolution
    conv1 = Conv2D(f1, (1,1), padding = 'same', activation='relu')(input_layer)
    
    # 3x3 convolution
    conv3 = Conv2D(f2_in, (1,1), padding = 'same', activation='relu')(input_layer)
    conv3 = Conv2D(f2_out, (3,3), padding='same', activation='relu')(conv3)
    
    # 5x5 convolution
    conv5 = Conv2D(f3_in, (1,1), padding='same', activation='relu')(input_layer)
    conv5 = Conv2D(f3_out, (5,5), padding='same', activation='relu')(conv5)
    
    # 3x3 max pooling
    pool = MaxPool2D((3,3), strides=(1,1), padding='same')(input_layer)
    pool = Conv2D(f4_out, (1,1), padding='same', activation='relu')(pool)
    
    output_layer = concatenate([conv1, conv3, conv5, pool])
    return output_layer

visible = Input(shape=(256,256,3))
layer = inception_module(visible, 64, 96, 128, 16, 32, 32)
layer = inception_module(layer, 128, 128, 192, 32, 96, 64)

model = Model(inputs=visible, outputs=layer)
model.summary()
plot_model(model, show_shapes=True, to_file='inception_module.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_42 (Conv2D)              (None, 256, 256, 96) 384         input_14[0][0]                   
__________________________________________________________________________________________________
conv2d_44 (Conv2D)              (None, 256, 256, 16) 64          input_14[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_15 (MaxPooling2D) (None, 256, 256, 3)  0           input_14[0][0]                   
__________________________________________________________________________________________________
conv2d_41 

## Residual Layer

In [27]:
from keras.layers import add, Activation

# function for creating an identity or projection residual module
def residual_module(layer_in, n_filters):
    merge_input = layer_in
    # check if the number of filters needs to be increase, assumes channels last format
    if layer_in.shape[-1] != n_filters:

        merge_input = Conv2D(n_filters, (1,1), padding= 'same' , activation= 'relu' ,
        kernel_initializer= 'he_normal' )(layer_in)
        # conv1
        conv1 = Conv2D(n_filters, (3,3), padding= 'same' , activation= 'relu' ,
        kernel_initializer= 'he_normal')(layer_in)
        # conv2
        conv2 = Conv2D(n_filters, (3,3), padding= 'same' , activation= 'linear' ,
        kernel_initializer= 'he_normal' )(conv1)
        # add filters, assumes filters/channels last
        layer_out = add([conv2, merge_input])
        # activation function
        layer_out = Activation( 'relu' )(layer_out)
        return layer_out
# define model input
visible = Input(shape=(256, 256, 3))
# add vgg module
layer = residual_module(visible, 64)
# create model
model = Model(inputs=visible, outputs=layer)
# summarize model
model.summary()
# plot model architecture
plot_model(model, show_shapes=True, to_file= 'residual_module.png' )

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_60 (Conv2D)              (None, 256, 256, 64) 1792        input_17[0][0]                   
__________________________________________________________________________________________________
conv2d_61 (Conv2D)              (None, 256, 256, 64) 36928       conv2d_60[0][0]                  
__________________________________________________________________________________________________
conv2d_59 (Conv2D)              (None, 256, 256, 64) 256         input_17[0][0]                   
__________________________________________________________________________________________________
add_2 (Add